# ЛР5. Логистическая регрессия

In [1]:
!pip install googledrivedownloader

ERROR: Could not find a version that satisfies the requirement google_drive_downloader (from versions: none)
ERROR: No matching distribution found for google_drive_downloader


In [2]:
import numpy as np
from scipy.special import expit
import pandas as pd

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from google_drive_downloader import GoogleDriveDownloader as gdd

## Задание 1.

Реализуйте собственные методы для классификации данных методом логистической регрессии. Для минимизации функции потерь используйте градиентный спуск.

$\frac{\delta J(\theta)}{\delta\theta}=\frac{1}{n} X^{T}[h(X)-y])$



In [3]:
class LogReg(BaseEstimator, ClassifierMixin):
    def __init__(self, bias=True, epoch=500000, lr = 1):
        self.bias = bias
        self.epoch = epoch
        self.lr = lr

    @staticmethod
    def prediction(X, w):
        # proba = 1 / (1 + np.exp(-(X @ w)))
        proba = expit(X @ w)
        return proba

    @staticmethod
    def classification(proba):
        vfunc = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
        return vfunc(proba)

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.cost = np.array([])
        
        if self.bias:
            self.X = np.concatenate((self.X, np.ones(self.X.shape[0]).reshape(-1, 1)), axis = 1)

        self.w = np.zeros(self.X.shape[1])
        self.score = []
        for epoch in tqdm(range(1, self.epoch)):
            pred = self.prediction(self.X, self.w)
            if epoch % 100 == 0:
                self.score.append(roc_auc_score(self.y, pred))
            self.w -= self.lr / self.X.shape[0] * (self.X.T @ (pred - self.y))
    
    def get_w(self):
        return self.w
    
    def get_cost(self):
        return self.cost

    def get_score(self):
        return self.score

    def predict(self, X):
        if self.bias:
            X = np.concatenate((X, np.ones(X.shape[0]).reshape(-1, 1)), axis = 1)
            out = self.prediction(X, self.w)
        else:
            out = self.prediction(X, self.w)
        return out

## Задание 2. 

Для каждого из подзаданий:

1. Проведите предварительную обработку данных.
2. Постройте модель классификации на основе метода логистической регрессии реализованной самостоятельно. 
3. Графически изобразите результаты классификации. Изобразите разделяющую гиперплоскость или кривую.
4. Подберите оптимальные гиперпараметры модели используя различные оценки.
5. Сделайте выводы о точности моделей.
6. Постройте несколько валиадционных кривых.


### 2.1 Скоринг отказа от услуг оператора связи.

Используйте данные из файла telecom.csv для предсказания отказа от услуг связи. Столбец "churn". 
Предврительно изучите и подготовьте данные.

In [4]:
gdd.download_file_from_google_drive(file_id='1KMOOvHfoltYupUsaS9nD2ltF2tkPrKze',
                                dest_path='./data/telecom.csv')

In [5]:
clf = LogReg(lr=0.1, epoch=5000)

In [6]:
df = pd.read_csv('telecom.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'telecom.csv'

In [ ]:
cat_columns = ['state', 'international plan', 'voice mail plan']
num_columns = [col for col in df.columns if col not in cat_columns + ['churn', 'phone number']]

In [ ]:
X = df.drop(cat_columns + ['phone number', 'churn'], axis=1)
y = df['churn']

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
mesh_size = 2
margin = 2

x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin

xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)

In [ ]:
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y)
fig.add_trace(go.Contour(x=xrange, y=yrange, z=Z, colorscale='RdBu'))

fig.show()

### 2.2 Предсказание диабета у пациентов.

Используйте данные из файла diabetes.csv для предсказания исхода для пациентов. Столбец "outcome". Предварительно изучите и подготовьте данные. 

In [ ]:
gdd.download_file_from_google_drive(file_id='1bme81mO6aYJNdNPcgeMyvwStp5g1iK47',
                                dest_path='./data/diabetes.csv')

In [ ]:
clf = make_pipeline(StandardScaler(), LogReg(lr=0.1, epoch=5000))
#clf = LogisticRegression(C=1e5)

In [ ]:
df = pd.read_csv('./data/diabetes.csv')

In [ ]:
X, y = df.drop(['Outcome'], axis=1), df['Outcome']

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
mesh_size = 1
margin = 1

x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin

xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)

In [ ]:
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y)
fig.add_trace(go.Contour(x=xrange, y=yrange, z=Z, colorscale='RdBu'))

fig.show()

In [ ]:
from sklearn.model_selection import learning_curve

## Задание 3. 

Для каждого из подзаданий:

1. Проведите предварительную обработку данных.
2. Постройте модель классификации на основе метода логистической регрессии из встроенной библиотеки. 
3. Графически изобразите результаты классификации. Изобразите разделяющую гиперплоскость или кривую.
4. Подберите оптимальные гиперпараметры модели используя различные оценки.
5. Сделайте выводы о точности моделей.
6. Постройте несколько валиадционных кривых и кривых обчуения. Поясните в текстовой ячейке выводы о работе моделей. 
7. Проведите регуляризацию. Определите оптимальный коэффициент регуляризации. 


### 3.1 Скоринг отказа от услуг оператора связи.

Используйте данные из файла telecom.csv для предсказания отказа от услуг связи. Столбец "churn". 
Предврительно изучите и подготовьте данные.

In [ ]:
clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [ ]:
df = pd.read_csv('./data/telecom.csv')

In [ ]:
cat_columns = ['state', 'international plan', 'voice mail plan']
num_columns = [col for col in df.columns if col not in cat_columns + ['churn', 'phone number']]

In [ ]:
X = df.drop(cat_columns + ['phone number', 'churn'], axis=1)
y = df['churn']

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
mesh_size = 2
margin = 2

x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin

xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)

In [ ]:
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y)
fig.add_trace(go.Contour(x=xrange, y=yrange, z=Z, colorscale='RdBu'))

fig.show()

### 3.2 Предсказание диабета у пациентов.

Используйте данные из файла diabetes.csv для предсказания исхода для пациентов. Столбец "outcome". Предварительно изучите и подготовьте данные. 

### 3.3 Данные теста микрочипов

Используйте данные из файла microchip_tests.txt для предсказания брака микрочипов. Предварительно изучите и подготовьте данные. 

### 3.4 ROC-AUC

На примере одного из классификаторов, постройте ROC кривые и оцените качество различных эстиматоров.Сделайте выводы.

## Ссылки

https://machinelearningmastery.com/logistic-regression-tutorial-for-machine-learning/

https://www.justintodata.com/logistic-regression-for-machine-learning-tutorial/

https://loginom.ru/blog/logistic-regression-roc-auc